In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import winsound
from collections import Counter
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [2]:
dir_name = 'C:/Users/hangbin/Google Drive/_New/Research/DeepHGLM/Simulation/data/'
data_type_list = [
    '20-500-2-logN',
    '1000-10-0.5-gamma', '100-100-0.5-gamma', '20-500-0.5-gamma',
    '1000-10-0-fixed', '100-100-0-fixed', '20-500-0-fixed',
]
# data_type_list = [
#     '1000-10-2-gamma', '100-100-2-gamma', '20-500-2-gamma', 
#     '1000-10-0.5-logN','100-100-0.5-logN','20-500-0.5-logN',
#     '1000-10-2-logN',  '100-100-2-logN',  '20-500-2-logN'
# ]

In [3]:
for data_type in data_type_list:
    
    settings = pd.read_csv(dir_name+'simul-settings-'+data_type+'.csv')

    for i in range(len(settings.columns)):
        item = settings.columns[i]
        if i < 5:
            exec(item+'=np.int32(settings["'+item+'"][0])')
        else:
            exec(item+'=settings["'+item+'"][0]')

    n_num_train = round(n_num*0.6)
    n_num_valid = round(n_num*0.2)
    n_num_test  = n_num - n_num_train - n_num_valid
    
    # set hyper-parameters

    lam_init = 1.
    if lam_init == lam: print("lam_init is same with ture lam !!!")

    num_iter = 1000
    num_epoch = 500
    lr = 0.001
    threshold = 0.001
    patience = 10
    penalty = 10

    callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]
    optimizer = Adam(learning_rate=lr)

    # 0 = pearson, 1 = rmse(y), 2 = rmse(mu), 3 = rmse(log mu)
    # 4 = pearson_new, 5 = rmse_new(y), 6 = rmse_new(mu), 7 = rmse(log mu)
    # 8 = rmse(u), 9 = lambda

    model_names = ['N0', 'N1', 'N2', 'M0', 'M1', 'M2', 'M3']

    N0_res = np.zeros((n_simul, 10))
    N1_res = np.zeros((n_simul, 10))
    N2_res = np.zeros((n_simul, 10))
    M0_res = np.zeros((n_simul, 10))
    M1_res = np.zeros((n_simul, 10))
    M2_res = np.zeros((n_simul, 10))
    M3_res = np.zeros((n_simul, 10))
        
    for simul_num in range(n_simul):

        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print('Time: ', current_time, ' simul_num: ', simul_num)

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_new = pd.read_csv(file_name+'-new.csv')

        # Split data into train, valid, and test set

        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]

        subset_names = ['_train', '_valid', '_test', '_new']
        for subset in subset_names:

            exec('temp_data = data'+subset)        
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]])')
            exec('y'+subset+'= np.array(temp_data["y"])')        
            exec('mu'+subset+'= np.array(temp_data["mu"])')
            exec('sub'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('num'+subset+'= np.array(temp_data["num"].astype("int32"))')

            if subset!='_new':
                exec('Z_sub'+subset+'= np.eye(n_sub)[sub'+subset+']')
                exec('N'+subset+'=n_sub*n_num'+subset)
            else:
                exec('Z_sub'+subset+'= np.zeros(shape=(n_new, n_sub))')
                exec('N'+subset+'=n'+subset)

            exec('dummy'+subset+'= np.zeros((N'+subset+',))')

        u = np.array([data['u'][n_num*i] for i in range(n_sub)])
        u_new = np.array(data_new['u'])    
        batch_size = len(y_train)

        ### N0 : Poi-NN ###

        lam_sub_N0 = 0
        tf.random.set_seed(simul_num)

        input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
        input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
        true_y = Input(shape=(1,), dtype='float32')

        m  = Dense(10, activation='leaky_relu')(input_X)
        m  = Dense(10, activation='leaky_relu')(m)        
        m  = Dense(10, activation='leaky_relu')(m)
        xb = Dense(1, activation='linear')(m)

        N0 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=xb)
        loss_poi = - K.sum(true_y*(xb)-K.exp(xb))
        N0.add_loss(loss_poi)
        N0.compile(optimizer=optimizer)
        history_N0 = N0.fit(
            [X_train, Z_sub_train, y_train], dummy_train, 
            epochs=num_epoch, batch_size=batch_size, verbose=0, callbacks=callbacks,
            validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid)
        )

        ### N1 : Poi-NN  with fixed vi (added to input) ###

        lam_sub_N1 = 0
        tf.random.set_seed(simul_num)    

        input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
        input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
        true_y = Input(shape=(1,), dtype='float32')

        XZ = Concatenate()([input_X, input_Z_sub])
        m  = Dense(10, activation='leaky_relu')(XZ)
        m  = Dense(10, activation='leaky_relu')(m)        
        m  = Dense(10, activation='leaky_relu')(m)
        xb = Dense(1, activation='linear')(m)

        N1 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=xb)
        loss_poi = - K.sum(true_y*(xb)-K.exp(xb))
        N1.add_loss(loss_poi)
        N1.compile(optimizer=optimizer)
        history_N1 = N1.fit(
            [X_train, Z_sub_train, y_train], dummy_train, 
            epochs=num_epoch, batch_size=batch_size, verbose=0, callbacks=callbacks,
            validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid)
        )

        ### N2 : Poi-NN  with fixed vi (added to last layer) ###

        lam_sub_N2 = 0
        tf.random.set_seed(simul_num)

        input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
        input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
        true_y = Input(shape=(1,), dtype='float32')

        m  = Dense(10, activation='leaky_relu')(input_X)
        m  = Dense(10, activation='leaky_relu')(m)        
        m  = Dense(10, activation='leaky_relu')(m)
        m  = Concatenate()([m, input_Z_sub])
        xb = Dense(1, activation='linear', use_bias=False)(m)

        N2 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=xb)
        loss_poi = - K.sum(true_y*(xb)-K.exp(xb))
        N2.add_loss(loss_poi)
        N2.compile(optimizer=optimizer)
        history_N2 = N2.fit(
            [X_train, Z_sub_train, y_train], dummy_train, 
            epochs=num_epoch, batch_size=batch_size, verbose=0, callbacks=callbacks,
            validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid)
        )

        ### M0 : Poi-gam-NN without rand constraint, without lambda adj ###

        lam_sub_M0 = lam_init
        for iteration in range(num_iter):

            ### STEP 1 : FIT NEURAL NETWORK

            tf.random.set_seed(simul_num)

            input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
            input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
            true_y = Input(shape=(1,), dtype='float32')

            m  = Dense(10, activation='leaky_relu')(input_X)
            m  = Dense(10, activation='leaky_relu')(m)        
            m  = Dense(10, activation='leaky_relu')(m)
            xb = Dense(1, activation='linear')(m)
            v_sub = Dense(1, activation='linear', use_bias=False)(input_Z_sub)    

            M0 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=[xb, v_sub])
            loss_poi_gam = (
                - K.sum(true_y*(xb+v_sub)-K.exp(xb+v_sub))
                - (1 / (K.ones((1, K.shape(true_y)[0]))@input_Z_sub)) @ K.transpose(input_Z_sub) @ (v_sub-K.exp(v_sub))/lam_sub_M0
            )
            M0.add_loss(loss_poi_gam)        
            M0.compile(optimizer=optimizer)
            if iteration!=0:
                M0.set_weights(wts)
            history_M0 = M0.fit([X_train, Z_sub_train, y_train], dummy_train, 
                                epochs=num_epoch, batch_size=batch_size, verbose=0,
                                callbacks=callbacks,
                                validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid))

            ### STEP 2 : FIND LAMBDA

            wts = M0.get_weights()
            u_sub_pred = np.exp(wts[-1])
            lam_sub_new = np.var(u_sub_pred, ddof=1)

            if abs(lam_sub_new-lam_sub_M0) > threshold:
                lam_sub_M0 = float(lam_sub_new)        
            else:
                lam_sub_M0 = float(lam_sub_new)        
                break

        ### M1 : Poi-gam-NN without rand constraint, with lambda adj ###

        lam_sub_M1 = lam_init
        for iteration in range(num_iter):

            ### STEP 1 : FIT NEURAL NETWORK

            tf.random.set_seed(simul_num)

            input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
            input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
            true_y = Input(shape=(1,), dtype='float32')

            m  = Dense(10, activation='leaky_relu')(input_X)
            m  = Dense(10, activation='leaky_relu')(m)        
            m  = Dense(10, activation='leaky_relu')(m)
            xb = Dense(1, activation='linear')(m)
            v_sub = Dense(1, activation='linear', use_bias=False)(input_Z_sub)    

            M1 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=[xb, v_sub])
            loss_poi_gam = (
                - K.sum(true_y*(xb+v_sub)-K.exp(xb+v_sub))
                - (1 / (K.ones((1, K.shape(true_y)[0]))@input_Z_sub)) @ K.transpose(input_Z_sub) @ (v_sub-K.exp(v_sub))/lam_sub_M1
            )
            M1.add_loss(loss_poi_gam)    
            M1.compile(optimizer=optimizer)
            if iteration!=0:
                M1.set_weights(wts)
            history_M1 = M1.fit([X_train, Z_sub_train, y_train], dummy_train, 
                                epochs=num_epoch, batch_size=batch_size, verbose=0,
                                callbacks=callbacks,
                                validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid))

            ### STEP 2 : FIND LAMBDA       

            wts = M1.get_weights()
            output = M1.predict([X_train, Z_sub_train, y_train])
            mu_sum_sub = Z_sub_train.T @ np.exp(output[0]) # np.exp(output[0]) is marginal mu_hat        
            u_sub_pred = np.exp(wts[-1])

            lam_sub_new = np.sum((u_sub_pred-1)**2)/n_sub*(
                0.5+ np.sqrt(0.25 + n_sub*np.sum((u_sub_pred-1)**2/mu_sum_sub)/(np.sum((u_sub_pred-1)**2))**2)
            )
            if abs(lam_sub_new-lam_sub_M1) > threshold:
                lam_sub_M1 = float(lam_sub_new)        
            else:
                lam_sub_M1 = float(lam_sub_new)
                break

        ### M2 : Poi-gam-NN with rand constraint, without lambda adj ###

        lam_sub_M2 = lam_init
        for iteration in range(num_iter):

            ### STEP 1 : FIT NEURAL NETWORK\

            tf.random.set_seed(simul_num)

            input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
            input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
            true_y = Input(shape=(1,), dtype='float32')

            m  = Dense(10, activation='leaky_relu')(input_X)
            m  = Dense(10, activation='leaky_relu')(m)        
            m  = Dense(10, activation='leaky_relu')(m)
            xb = Dense(1, activation='linear')(m)
            v_sub = Dense(1, activation='linear', use_bias=False)(input_Z_sub)    

            M2 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=[xb, v_sub])
            loss_poi_gam = (
                - K.sum(true_y*(xb+v_sub)-K.exp(xb+v_sub))
                - (1 / (K.ones((1, K.shape(true_y)[0]))@input_Z_sub)) @ K.transpose(input_Z_sub) @ (v_sub-K.exp(v_sub))/lam_sub_M2
            )
            M2.add_loss(loss_poi_gam)        
            M2.compile(optimizer=optimizer)
            if iteration!=0:
                M2.set_weights(wts)
            history_M2 = M2.fit([X_train, Z_sub_train, y_train], dummy_train, 
                                epochs=num_epoch, batch_size=batch_size, verbose=0,
                                callbacks=callbacks,
                                validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid))

            ### STEP 2 : FIND LAMBDA       

            wts = M2.get_weights()
            v_sub_adj = - np.log(np.mean(np.exp(wts[-1])))
            wts[-2] = wts[-2] - v_sub_adj
            wts[-1] = wts[-1] + v_sub_adj
            M2.set_weights(wts)
            u_sub_pred = np.exp(wts[-1])    

            lam_sub_new = np.var(u_sub_pred, ddof=1)
            if abs(lam_sub_new-lam_sub_M2) > threshold:
                lam_sub_M2 = float(lam_sub_new)        
            else:
                lam_sub_M2 = float(lam_sub_new) 
                break            

        ### M3 : Poi-gam-NN with rand constraint, with lambda adj ###

        lam_sub_M3 = lam_init
        for iteration in range(num_iter):

            ### STEP 1 : FIT NEURAL NETWORK

            tf.random.set_seed(simul_num)

            input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
            input_Z_sub = Input(shape=(np.shape(Z_sub_train)[1],), dtype='float32')
            true_y = Input(shape=(1,), dtype='float32')

            m  = Dense(10, activation='leaky_relu')(input_X)
            m  = Dense(10, activation='leaky_relu')(m)        
            m  = Dense(10, activation='leaky_relu')(m)
            xb = Dense(1, activation='linear')(m)
            v_sub = Dense(1, activation='linear', use_bias=False)(input_Z_sub)    

            M3 = Model(inputs=[input_X, input_Z_sub, true_y], outputs=[xb, v_sub])
            loss_poi_gam = (
                - K.sum(true_y*(xb+v_sub)-K.exp(xb+v_sub))
                - (1 / (K.ones((1, K.shape(true_y)[0]))@input_Z_sub)) @ K.transpose(input_Z_sub) @ (v_sub-K.exp(v_sub))/lam_sub_M3
            )
            M3.add_loss(loss_poi_gam)
            M3.compile(optimizer=optimizer)
            if iteration!=0:
                M3.set_weights(wts)
            history_M3 = M3.fit([X_train, Z_sub_train, y_train], dummy_train, 
                                epochs=num_epoch, batch_size=batch_size, verbose=0,
                                callbacks=callbacks,
                                validation_data=([X_valid, Z_sub_valid, y_valid], dummy_valid))

            ### STEP 2 : FIND LAMBDA       

            wts = M3.get_weights()
            v_sub_adj = - np.log(np.mean(np.exp(wts[-1])))
            wts[-2] = wts[-2] - v_sub_adj
            wts[-1] = wts[-1] + v_sub_adj
            M3.set_weights(wts)
            output = M3.predict([X_train, Z_sub_train, y_train])
            mu_sum_sub = Z_sub_train.T @ np.exp(output[0]) # np.exp(output[0]) is marginal mu_hat
            u_sub_pred = np.exp(wts[-1])

            lam_sub_new = np.sum((u_sub_pred-1)**2)/n_sub*(
                0.5+ np.sqrt(0.25 + n_sub*np.sum((u_sub_pred-1)**2/mu_sum_sub)/(np.sum((u_sub_pred-1)**2))**2)
            )
            if abs(lam_sub_new-lam_sub_M3) > threshold:
                lam_sub_M3 = float(lam_sub_new)        
            else:
                lam_sub_M3 = float(lam_sub_new)
                break

        # make result frame

        for model_name in model_names:

            exec('output_test_'+model_name+' = '+model_name+'.predict([X_test, Z_sub_test, dummy_test])')
            exec('output_new_'+model_name+' = '+model_name+'.predict([X_new, Z_sub_new, dummy_new])')
            if model_name[0] == 'N':
                exec('mu_test_'+model_name+' = np.exp(output_test_'+model_name+')')
                exec('mu_new_'+model_name+' = np.exp(output_new_'+model_name+')')
            else:
                exec('mu_test_'+model_name+' = np.exp(output_test_'+model_name+'[0] + output_test_'+model_name+'[1])')
                exec('mu_new_'+model_name+' = np.exp(output_new_'+model_name+'[0] + output_new_'+model_name+'[1])')
            exec('mu_test_'+model_name+' = mu_test_'+model_name+'.T[0]')
            exec('mu_new_'+model_name+' = mu_new_'+model_name+'.T[0]') 

            exec(model_name+'_res[simul_num, 0] = np.sqrt(np.mean((y_test - mu_test_'+model_name+')**2/mu_test_'+model_name+'))')
            exec(model_name+'_res[simul_num, 1] = np.sqrt(np.mean((y_test - mu_test_'+model_name+')**2))')
            exec(model_name+'_res[simul_num, 2] = np.sqrt(np.mean((mu_test - mu_test_'+model_name+')**2))')
            exec(model_name+'_res[simul_num, 3] = np.sqrt(np.mean((np.log(mu_test) - np.log(mu_test_'+model_name+'))**2))')
            exec(model_name+'_res[simul_num, 4] = np.sqrt(np.mean((y_new - mu_new_'+model_name+')**2/mu_new_'+model_name+'))')
            exec(model_name+'_res[simul_num, 5] = np.sqrt(np.mean((y_new - mu_new_'+model_name+')**2))')
            exec(model_name+'_res[simul_num, 6] = np.sqrt(np.mean((mu_new - mu_new_'+model_name+')**2))')
            exec(model_name+'_res[simul_num, 7] = np.sqrt(np.mean((np.log(mu_new) - np.log(mu_new_'+model_name+'))**2))')

            if model_name[0] == 'M':
                exec('u_pred_'+model_name+' = np.exp('+model_name+'.get_weights()[-1].T[0])')
                exec(model_name+'_res[simul_num, 8] = np.sqrt(np.mean((u - u_pred_'+model_name+')**2))')
                exec(model_name+'_res[simul_num, 9] = lam_sub_'+model_name)
            elif model_name == 'N2':
                exec('u_pred_'+model_name+' = np.exp('+model_name+'.get_weights()[-1].T[0][10:])')
                exec(model_name+'_res[simul_num, 8] = np.sqrt(np.mean((u - u_pred_'+model_name+')**2))')
                exec(model_name+'_res[simul_num, 9] = lam_sub_'+model_name)
                
    res_colnames = ['RMSE_p', 'RMSE_y', 'RMSE_mu', 'RMSE_log_mu', 
                    'RMSE_p_new', 'RMSE_y_new', 'RMSE_mu_new', 'RMSE_log_mu_new', 
                    'RMSE_u', 'lambda']

    sample_mu_test = np.zeros((N_test, len(model_names)))
    sample_mu_new = np.zeros((N_new, len(model_names)))

    for i in range(len(model_names)):
        model_name = model_names[i]
        exec(model_name+'_res = pd.DataFrame('+model_name+'_res, columns=res_colnames)')    
        exec(model_name+'_res.to_csv(dir_name+data_type+"-"+model_name+".csv", index=False)')
        exec('sample_mu_test[:,'+str(i)+'] = mu_test_'+model_name)
        exec('sample_mu_new[:,'+str(i)+'] = mu_new_'+model_name)

    sample_mu_test = pd.DataFrame(sample_mu_test, columns=model_names)
    sample_mu_test.to_csv(dir_name+'sample-mu-'+data_type+'.csv')
    sample_mu_new = pd.DataFrame(sample_mu_new, columns=model_names)
    sample_mu_new.to_csv(dir_name+'sample-mu-'+data_type+'-new.csv')
                

winsound.Beep(440, 1000)   

Time:  17:19:30  simul_num:  0
Time:  17:22:19  simul_num:  1
Time:  17:25:16  simul_num:  2
Time:  17:28:15  simul_num:  3
Time:  17:31:08  simul_num:  4
Time:  17:34:30  simul_num:  5
Time:  17:38:00  simul_num:  6
Time:  17:41:13  simul_num:  7
Time:  17:44:29  simul_num:  8
Time:  17:47:18  simul_num:  9
Time:  17:51:03  simul_num:  10
Time:  17:53:18  simul_num:  11
Time:  17:56:47  simul_num:  12
Time:  17:59:53  simul_num:  13
Time:  18:03:00  simul_num:  14
Time:  18:06:18  simul_num:  15
Time:  18:08:35  simul_num:  16
Time:  18:12:38  simul_num:  17
Time:  18:17:44  simul_num:  18
Time:  18:20:50  simul_num:  19
Time:  18:24:36  simul_num:  20
Time:  18:27:41  simul_num:  21
Time:  18:32:32  simul_num:  22
Time:  18:36:26  simul_num:  23
Time:  18:39:43  simul_num:  24
Time:  18:42:56  simul_num:  25
Time:  18:47:09  simul_num:  26
Time:  18:50:22  simul_num:  27
Time:  18:54:00  simul_num:  28
Time:  18:56:57  simul_num:  29
Time:  19:00:54  simul_num:  30
Time:  19:05:28  s

Time:  05:07:11  simul_num:  57
Time:  05:09:23  simul_num:  58
Time:  05:11:24  simul_num:  59
Time:  05:13:18  simul_num:  60
Time:  05:15:25  simul_num:  61
Time:  05:17:46  simul_num:  62
Time:  05:19:38  simul_num:  63
Time:  05:21:37  simul_num:  64
Time:  05:24:04  simul_num:  65
Time:  05:26:05  simul_num:  66
Time:  05:28:20  simul_num:  67
Time:  05:30:30  simul_num:  68
Time:  05:32:20  simul_num:  69
Time:  05:34:21  simul_num:  70
Time:  05:36:32  simul_num:  71
Time:  05:38:31  simul_num:  72
Time:  05:41:04  simul_num:  73
Time:  05:43:08  simul_num:  74
Time:  05:44:55  simul_num:  75
Time:  05:47:23  simul_num:  76
Time:  05:49:19  simul_num:  77
Time:  05:51:19  simul_num:  78
Time:  05:53:09  simul_num:  79
Time:  05:55:11  simul_num:  80
Time:  05:57:30  simul_num:  81
Time:  05:59:23  simul_num:  82
Time:  06:01:45  simul_num:  83
Time:  06:04:08  simul_num:  84
Time:  06:07:06  simul_num:  85
Time:  06:09:13  simul_num:  86
Time:  06:11:05  simul_num:  87
Time:  0

Time:  12:47:47  simul_num:  14
Time:  12:48:41  simul_num:  15
Time:  12:50:00  simul_num:  16
Time:  12:51:30  simul_num:  17
Time:  12:52:43  simul_num:  18
Time:  12:54:10  simul_num:  19
Time:  12:55:39  simul_num:  20
Time:  12:56:49  simul_num:  21
Time:  12:57:48  simul_num:  22
Time:  12:58:54  simul_num:  23
Time:  13:00:39  simul_num:  24
Time:  13:01:35  simul_num:  25
Time:  13:02:42  simul_num:  26
Time:  13:03:50  simul_num:  27
Time:  13:05:13  simul_num:  28
Time:  13:06:52  simul_num:  29
Time:  13:08:09  simul_num:  30
Time:  13:09:47  simul_num:  31
Time:  13:10:36  simul_num:  32
Time:  13:11:56  simul_num:  33
Time:  13:12:57  simul_num:  34
Time:  13:14:01  simul_num:  35
Time:  13:14:54  simul_num:  36
Time:  13:15:44  simul_num:  37
Time:  13:16:40  simul_num:  38
Time:  13:17:34  simul_num:  39
Time:  13:18:17  simul_num:  40
Time:  13:19:35  simul_num:  41
Time:  13:20:52  simul_num:  42
Time:  13:22:14  simul_num:  43
Time:  13:23:07  simul_num:  44
Time:  1